In [1]:
import random
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

data = pd.read_csv("clean-news.csv",encoding = "ISO-8859-1")
data['text'] = data['title'] + data['text']
data = data.drop('title',1)
data.loc[(data['label'] == 'FAKE'),'label'] = 0
data.loc[(data['label'] == 'REAL'), 'label'] = 1


In [2]:
def preprocess(dataset):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    stop_words = set(stopwords.words('english'))
    lemmatizer = nltk.stem.WordNetLemmatizer()
    garbage = "~`!@#$%^&*()_-+={[}]|\:;'<,>.?/"
    
    for i in range(dataset.shape[0]):
        #print(i)
        string = str(dataset.iloc[i,0])
        #print('['+string+']')
        #tokenize
        string1 = tokenizer.tokenize(string)
        #remove stop word
        string2 = ([token.lower() for token in string1 \
                               if token not in stop_words])
        #normalize
        string3 = " ".join([lemmatizer.lemmatize(token) for token \
                                    in string2]).strip()
        #remove garbage
        dataset.loc[i,'text'] = "".join([char for char in string3 \
                                   if char not in garbage])
    #print(dataset.columns)
    #print(dataset.head(10))
    return dataset


In [3]:
data = pd.read_csv("clean-news.csv",encoding = "ISO-8859-1")
data['text'] = data['title'] + data['text']
data = data.drop('title',1)
data.loc[(data['label'] == 'FAKE'),'label'] = 0
data.loc[(data['label'] == 'REAL'), 'label'] = 1

In [4]:
data = data[data['text'].notna()]
data = data[data['label'].notna()]
data = preprocess(data)
data['label'].fillna(1, inplace=True)
data['text'].fillna('', inplace=True)

In [5]:
data.head()

,text,label
0,smell hillary feardaniel greenfield shillman j...,0
1,watch exact moment paul ryan committed politic...,0
2,kerry go paris gesture sympathyus secretary st...,1
3,bernie supporter twitter erupt anger dnc tried...,0
4,battle new york primary mattersit primary day ...,1


In [6]:
x = data.iloc[:,0].values
y = data.iloc[:,1].values

In [7]:
x[:2]

array(['smell hillary feardaniel greenfield shillman journalism fellow freedom center new york writer focusing radical islam final stretch election hillary rodham clinton gone war fbi word unprecedented thrown around often election ought retired still unprecedented nominee major political party go war fbi exactly hillary people done coma patient waking watching hour cnn hospital bed would assume fbi director james comey hillary opponent election fbi attack everyone obama cnn hillary people circulated letter attacking comey currently medium hit piece lambasting targeting trump surprising clinton ally start running attack ad fbi fbi leadership warned entire left wing establishment form lynch mob continue going hillary fbi credibility attacked medium democrat preemptively head result investigation clinton foundation hillary clinton covert struggle fbi agent obama doj people gone explosively public new york time compared comey j edgar hoover bizarre headline james comey role recall hoover 

In [8]:
td = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))
X = td.fit_transform(x).toarray()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,\
                                                    test_size = 0.2,\
                                                    random_state = 0)

In [10]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0, solver='lbfgs')
model = classifier.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model.predict_proba(X_test) #confident score a doc is [fake, real]

In [ ]:
type(model.coef_) #Coefficient of the features in the decision function.

In [ ]:
model.coef_.shape

In [ ]:
coef = model.coef_.reshape(-1)
coef.shape

In [ ]:
import numpy as np
np.argmax(coef)
words[np.argmax(coef)]